
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [4]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_2365109/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-26 01:43:32.736227: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 01:43:32.773227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 01:43:32.773278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 01:43:32.774392: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 01:43:32.780392: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 01:43:32.781655: I tensorflow/core/platform/cpu_feature_guard.cc:1

2024-03-26 01:43:34.312846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [7]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [8]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [9]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[15,7]],
            regularizers=[ 0.1,0.02],
            learning_rate=[ 0.1,0.02])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.1, learning_rate=0.1


+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.5397750419997465
Stopped at epoch:  10
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  1.0974060730004567
Stopped at epoch:  16
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.4474405339997247
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.7211028809997515
Stopped at epoch:  8
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.4713292639999054
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.3693751680002606
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.511557541000002
Stopped at epoch:  7
1/1 [==========

2024-03-26 01:43:46.356295: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  5.120642623000094
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 9 ++++++++++++++++++
Elapsed time:  0.03756991799946263
Stopped at epoch:  0
1/1 [==============================] - 0s 24ms/step
+++++++++++ [1_0] | 10 ++++++++++++++++++
Elapsed time:  0.04905479000080959
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_0] | 11 ++++++++++++++++++
Elapsed time:  0.049379905999558105
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 12 ++++++++++++++++++
Elapsed time:  0.036359810999783804
Stopped at epoch:  0
1/1 [==============================] - 0s 27ms/step
+++++++++++ [1_0] | 13 ++++++++++++++++++
Elapsed time:  0.03475543800050218
Stopped at epoch:  0
1/1 [==============================] - 0s 38ms/step
+++++++++++ [1_0] | 14 ++++++++++++++++++
Elapsed time:  0.05442133599990484
Stopped at epoch:  0
1/1 [==============================] - 0s 22m

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_0_0,0.812728,0.851815,0.598100,0.803866,0.207255,0.150030,0.151313,0.150633
model_2_0_1,0.816218,0.853029,0.599672,0.805076,0.203392,0.148800,0.150721,0.149704
model_2_0_2,0.833699,0.859221,0.595554,0.808448,0.184046,0.142530,0.152271,0.147114
model_2_0_3,0.846550,0.860141,0.578159,0.805077,0.169824,0.141599,0.158820,0.149703
model_2_0_4,0.855762,0.860610,0.596120,0.809547,0.159629,0.141125,0.152058,0.146270
model_2_0_9,0.861123,0.763049,0.402242,0.696733,0.153696,0.239900,0.225052,0.232913
model_2_0_5,0.862727,0.855751,0.631511,0.814321,0.151921,0.146044,0.138734,0.142604
model_2_0_6,0.863338,0.852466,0.633846,0.812567,0.151244,0.149370,0.137855,0.143951
model_2_0_8,0.864267,0.776612,0.500559,0.728879,0.150217,0.226168,0.188036,0.208224
model_2_0_7,0.866624,0.815574,0.596990,0.778318,0.147607,0.186721,0.151731,0.170255


...

DataFrame salvo em ../results/metrics_2_9
Testando combinacao3: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.1
+++++++++++ [3_0] | 1 ++++++++++++++++++
Elapsed time:  1.2596022550005728
Stopped at epoch:  14
1/1 [==============================] - 0s 17ms/step
+++++++++++ [3_0] | 2 ++++++++++++++++++
Elapsed time:  1.0336818860005224
Stopped at epoch:  17
1/1 [==============================] - 0s 19ms/step
+++++++++++ [3_0] | 3 ++++++++++++++++++
Elapsed time:  1.0817072379995807
Stopped at epoch:  15
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_0] | 4 ++++++++++++++++++
Elapsed time:  0.4475854319998689
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [3_0] | 5 ++++++++++++++++++
Elapsed time:  0.7563577070004612
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [3_0] | 6 ++++++++++++++++++
Elapsed time:  1.1233560609998676
Stopped at epoch:  16
1/1 [==============================] - 

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_8_3,0.800705,0.684625,-0.695115,0.638177,0.220561,0.306826,0.109547,0.213989
model_3_8_1,0.800728,0.683576,-0.637495,0.640227,0.220536,0.307846,0.105823,0.212776
model_3_8_2,0.801111,0.686179,-0.655572,0.641565,0.220112,0.305313,0.106991,0.211985
model_3_8_6,0.801170,0.688961,-0.769734,0.638116,0.220047,0.302607,0.114369,0.214025
model_3_8_4,0.801217,0.689515,-0.731925,0.640543,0.219995,0.302068,0.111926,0.212589
model_3_8_0,0.801261,0.686712,-0.614642,0.644133,0.219946,0.304795,0.104346,0.210466
model_3_8_5,0.801608,0.691423,-0.745247,0.641520,0.219561,0.300212,0.112787,0.212012
model_3_8_8,0.801763,0.692062,-0.768486,0.640881,0.219390,0.299590,0.114288,0.212389
model_3_8_9,0.801961,0.693441,-0.770743,0.641965,0.219171,0.298249,0.114434,0.211748
model_3_8_7,0.802323,0.696214,-0.770582,0.644389,0.218770,0.295551,0.114424,0.210315


DataFrame salvo em ../results/metrics_3_8
+++++++++++ [3_9] | 1 ++++++++++++++++++
Elapsed time:  0.5279044839999187
Stopped at epoch:  8
1/1 [==============================] - 0s 16ms/step
+++++++++++ [3_9] | 2 ++++++++++++++++++
Elapsed time:  0.43563758900018
Stopped at epoch:  9
1/1 [==============================] - 0s 16ms/step
+++++++++++ [3_9] | 3 ++++++++++++++++++
Elapsed time:  0.6903848050005763
Stopped at epoch:  11
1/1 [==============================] - 0s 25ms/step
+++++++++++ [3_9] | 4 ++++++++++++++++++
Elapsed time:  1.577757821999512
Stopped at epoch:  22
1/1 [==============================] - 0s 25ms/step
+++++++++++ [3_9] | 5 ++++++++++++++++++
Elapsed time:  0.4374919849997241
Stopped at epoch:  6
1/1 [==============================] - 0s 31ms/step
+++++++++++ [3_9] | 6 ++++++++++++++++++
Elapsed time:  0.5875566190006793
Stopped at epoch:  9
1/1 [==============================] - 0s 23ms/step
+++++++++++ [3_9] | 7 ++++++++++++++++++
Elapsed time:  0.4148518520005

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_9_19,0.800591,0.696404,-1.857247,0.621155,0.220687,0.400045,0.183364,0.298077
model_3_9_12,0.800603,0.697809,-1.716826,0.627790,0.220673,0.398194,0.174352,0.292857
model_3_9_21,0.800676,0.695737,-1.877004,0.619806,0.220593,0.400924,0.184632,0.299139
model_3_9_23,0.800932,0.698237,-1.935083,0.619793,0.220310,0.397630,0.188359,0.299149
model_3_9_22,0.801008,0.698038,-1.907191,0.620687,0.220225,0.397892,0.186569,0.298446
model_3_9_20,0.801261,0.698626,-1.856054,0.623171,0.219946,0.397118,0.183287,0.296491
model_3_9_9,0.801482,0.701689,-1.647981,0.633873,0.219701,0.393081,0.169934,0.288071
model_3_9_13,0.801624,0.701435,-1.714032,0.631112,0.219543,0.393416,0.174173,0.290243
model_3_9_8,0.801991,0.703737,-1.615527,0.636935,0.219138,0.390382,0.167851,0.285662
model_3_9_14,0.802003,0.702720,-1.725662,0.631806,0.219125,0.391723,0.174919,0.289698


DataFrame salvo em ../results/metrics_3_9
Testando combinacao4: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.02
+++++++++++ [4_0] | 1 ++++++++++++++++++
Elapsed time:  2.1111914520006394
Stopped at epoch:  27
1/1 [==============================] - 0s 18ms/step
+++++++++++ [4_0] | 2 ++++++++++++++++++
Elapsed time:  1.9060711649999575
Stopped at epoch:  27
1/1 [==============================] - 0s 17ms/step
+++++++++++ [4_0] | 3 ++++++++++++++++++
Elapsed time:  0.5383833249998133
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [4_0] | 4 ++++++++++++++++++
Elapsed time:  0.3798215839997283
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [4_0] | 5 ++++++++++++++++++
Elapsed time:  0.5138143450003554
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [4_0] | 6 ++++++++++++++++++
Elapsed time:  0.5244213269997999
Stopped at epoch:  6
1/1 [==============================] - 0